In [1]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
import numpy as np
from tqdm import tqdm
from warnings import warn

from aif360.datasets import BinaryLabelDataset
from aif360.datasets import StandardDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from aif360.algorithms.postprocessing.reject_option_classification\
        import RejectOptionClassification
from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider
import pandas as pd
import pickle
from sklearn.linear_model import Lasso

`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df

## Huangrui's Dataset 

In [2]:
# def load_preproc_data_german(protected_attributes=None):
#     """
#     Load and pre-process german credit dataset.
#     Args:
#         protected_attributes(list or None): If None use all possible protected
#             attributes, else subset the protected attributes to the list.

#     Returns:
#         GermanDataset: An instance of GermanDataset with required pre-processing.

#     """
#     def custom_preprocessing(df):
#         """ Custom pre-processing for German Credit Data
#         """

#         def group_credit_hist(x):
#             if x in ['A30', 'A31', 'A32']:
#                 return 'None/Paid'
#             elif x == 'A33':
#                 return 'Delay'
#             elif x == 'A34':
#                 return 'Other'
#             else:
#                 return 'NA'

#         def group_employ(x):
#             if x == 'A71':
#                 return 'Unemployed'
#             elif x in ['A72', 'A73']:
#                 return '1-4 years'
#             elif x in ['A74', 'A75']:
#                 return '4+ years'
#             else:
#                 return 'NA'

#         def group_savings(x):
#             if x in ['A61', 'A62']:
#                 return '<500'
#             elif x in ['A63', 'A64']:
#                 return '500+'
#             elif x == 'A65':
#                 return 'Unknown/None'
#             else:
#                 return 'NA'

#         def group_status(x):
#             if x in ['A22', 'A22']:
#                 return '<200'
#             elif x in ['A23']:
#                 return '200+'
#             elif x == 'A14':
#                 return 'None'
#             else:
#                 return 'NA'

#         status_map = {'A91': 1.0, 'A93': 1.0, 'A94': 1.0,
#                     'A92': 0.0, 'A95': 0.0}
#         df['sex'] = df['personal_status'].replace(status_map)
#         #drop personal_status,not leak sex information
#         df = df.drop('personal_status', axis=1)

#         # group credit history, savings, and employment
#         df['credit_history'] = df['credit_history'].apply(lambda x: group_credit_hist(x))
#         df['savings'] = df['savings'].apply(lambda x: group_savings(x))
#         df['employment'] = df['employment'].apply(lambda x: group_employ(x))
#         df['age'] = (df['age'] >= 26).astype(float)
#         df['status'] = df['status'].apply(lambda x: group_status(x))
#         df["Y"] = df["credit"].apply(lambda x: 1 if x == 1 else 0)
#         df.drop(["credit"], axis=1, inplace=True)
#         return df

#     # Feature partitions
#     D_features = ['sex', 'age'] if protected_attributes is None else protected_attributes
#     Y_features = ['Y']

#     # privileged classes
#     all_privileged_classes = {"sex": [1.0],
#                               "age": [1.0]}

#     # protected attribute maps
#     all_protected_attribute_maps = {"sex": {1.0: 'Male', 0.0: 'Female'},
#                                     "age": {1.0: 'Old', 0.0: 'Young'}}

#     return GermanDataset(
#         label_name=Y_features[0],
#         favorable_classes=[1],
#         protected_attribute_names=D_features,
#         privileged_classes=[all_privileged_classes[x] for x in D_features],
#         instance_weights_name=None,
#         features_to_keep=[],
#         metadata={ 'label_maps': [{1.0: 'Good Credit', 0: 'Bad Credit'}],
#                    'protected_attribute_maps': [all_protected_attribute_maps[x]
#                                 for x in D_features]},
#         custom_preprocessing=custom_preprocessing)


In [3]:
class GermanDataset(StandardDataset):
    """German credit Dataset.

    See :file:`aif360/data/raw/german/README.md`.
    """

    def __init__(self, path, label_name='Y', favorable_classes=[1],
                 protected_attribute_names=['sex', 'age'],
                 privileged_classes=[[1],[1]],
                 instance_weights_name=None,
                 categorical_features=[],
                 features_to_keep=[], features_to_drop=[],
                 na_values=[], custom_preprocessing=None,
                 metadata=None):
        
        df = pd.read_csv(path)
        

        super(GermanDataset, self).__init__(df=df, label_name=label_name,
            favorable_classes=favorable_classes,
            protected_attribute_names=protected_attribute_names,
            privileged_classes=privileged_classes,
            instance_weights_name=instance_weights_name,
            categorical_features=categorical_features,
            features_to_keep=features_to_keep,
            features_to_drop=features_to_drop, na_values=na_values,
            custom_preprocessing=custom_preprocessing, metadata=metadata)

#### Load dataset and specify options

In [4]:
## import dataset
dataset_used = "german" # "german", "german", "compas"
protected_attribute_used = 2 # 1, 2


#     dataset_orig = GermanDataset()
if protected_attribute_used == 1:
    privileged_groups = [{'sex': 1}]
    unprivileged_groups = [{'sex': 0}]
else:
    privileged_groups = [{'age': 1}]
    unprivileged_groups = [{'age': 0}]
        
# Metric used (should be one of allowed_metrics)
metric_name = "Equal opportunity difference"

# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05
        
        
#random seed for calibrated equal odds prediction
random_seed = 12345679
np.random.seed(random_seed)

# Verify metric name
allowed_metrics = ["Statistical parity difference",
                   "Average odds difference",
                   "Equal opportunity difference"]
if metric_name not in allowed_metrics:
    raise ValueError("Metric name should be one of allowed metrics")

#### Split into train, test and validation

In [5]:
# # 5 fold cross validation
# Z =  dataset_orig.split(5, shuffle=True,seed = random_seed)
# # i th fold
# dataset_train1 = Z[0].copy()
# dataset_train1.features = np.concatenate((Z[0].features,Z[1].features,Z[2].features,Z[3].features),axis=0)
# dataset_train1.scores = np.concatenate((Z[0].scores,Z[1].scores,Z[2].scores,Z[3].scores),axis=0)
# dataset_train1.labels = np.concatenate((Z[0].labels,Z[1].labels,Z[2].labels,Z[3].labels),axis=0)
# dataset_train1.protected_attributes = np.concatenate((Z[0].protected_attributes,Z[1].protected_attributes,Z[2].protected_attributes,Z[3].protected_attributes),axis=0)
# dataset_train1.instance_weights = np.concatenate((Z[0].instance_weights,Z[1].instance_weights,Z[2].instance_weights,Z[3].instance_weights),axis=0)
# dataset_train1.instance_names = np.concatenate((Z[0].instance_names,Z[1].instance_names,Z[2].instance_names,Z[3].instance_names),axis=0)
# dataset_train1.metadata = Z[0].metadata.copy()
# dataset_test1= Z[4].copy()

# dataset_train2 = Z[1].copy()
# dataset_train2.features = np.concatenate((Z[1].features,Z[2].features,Z[3].features,Z[4].features),axis=0)
# dataset_train2.scores = np.concatenate((Z[1].scores,Z[2].scores,Z[3].scores,Z[4].scores),axis=0)
# dataset_train2.labels = np.concatenate((Z[1].labels,Z[2].labels,Z[3].labels,Z[4].labels),axis=0)
# dataset_train2.protected_attributes = np.concatenate((Z[1].protected_attributes,Z[2].protected_attributes,Z[3].protected_attributes,Z[4].protected_attributes),axis=0)
# dataset_train2.instance_weights = np.concatenate((Z[1].instance_weights,Z[2].instance_weights,Z[3].instance_weights,Z[4].instance_weights),axis=0)
# dataset_train2.instance_names = np.concatenate((Z[1].instance_names,Z[2].instance_names,Z[3].instance_names,Z[4].instance_names),axis=0)
# dataset_train2.metadata = Z[1].metadata.copy()
# dataset_test2= Z[0].copy()

# dataset_train3 = Z[2].copy()
# dataset_train3.features = np.concatenate((Z[2].features,Z[3].features,Z[4].features,Z[0].features),axis=0)
# dataset_train3.scores = np.concatenate((Z[2].scores,Z[3].scores,Z[4].scores,Z[0].scores),axis=0)
# dataset_train3.labels = np.concatenate((Z[2].labels,Z[3].labels,Z[4].labels,Z[0].labels),axis=0)
# dataset_train3.protected_attributes = np.concatenate((Z[2].protected_attributes,Z[3].protected_attributes,Z[4].protected_attributes,Z[0].protected_attributes),axis=0)
# dataset_train3.instance_weights = np.concatenate((Z[2].instance_weights,Z[3].instance_weights,Z[4].instance_weights,Z[0].instance_weights),axis=0)
# dataset_train3.instance_names = np.concatenate((Z[2].instance_names,Z[3].instance_names,Z[4].instance_names,Z[0].instance_names),axis=0)
# dataset_train3.metadata = Z[2].metadata.copy()
# dataset_test3= Z[1].copy()

# dataset_train4 = Z[3].copy()
# dataset_train4.features = np.concatenate((Z[3].features,Z[4].features,Z[0].features,Z[1].features),axis=0)
# dataset_train4.scores = np.concatenate((Z[3].scores,Z[4].scores,Z[0].scores,Z[1].scores),axis=0)
# dataset_train4.labels = np.concatenate((Z[3].labels,Z[4].labels,Z[0].labels,Z[1].labels),axis=0)
# dataset_train4.protected_attributes = np.concatenate((Z[3].protected_attributes,Z[4].protected_attributes,Z[0].protected_attributes,Z[1].protected_attributes),axis=0)
# dataset_train4.instance_weights = np.concatenate((Z[3].instance_weights,Z[4].instance_weights,Z[0].instance_weights,Z[1].instance_weights),axis=0)
# dataset_train4.instance_names = np.concatenate((Z[3].instance_names,Z[4].instance_names,Z[0].instance_names,Z[1].instance_names),axis=0)
# dataset_train4.metadata = Z[3].metadata.copy()
# dataset_test4= Z[2].copy()

# dataset_train5 = Z[4].copy()
# dataset_train5.features = np.concatenate((Z[4].features,Z[0].features,Z[1].features,Z[2].features),axis=0)
# dataset_train5.scores = np.concatenate((Z[4].scores,Z[0].scores,Z[1].scores,Z[2].scores),axis=0)
# dataset_train5.labels = np.concatenate((Z[4].labels,Z[0].labels,Z[1].labels,Z[2].labels),axis=0)
# dataset_train5.protected_attributes = np.concatenate((Z[4].protected_attributes,Z[0].protected_attributes,Z[1].protected_attributes,Z[2].protected_attributes),axis=0)
# dataset_train5.instance_weights = np.concatenate((Z[4].instance_weights,Z[0].instance_weights,Z[1].instance_weights,Z[2].instance_weights),axis=0)
# dataset_train5.instance_names = np.concatenate((Z[4].instance_names,Z[0].instance_names,Z[1].instance_names,Z[2].instance_names),axis=0)
# dataset_train5.metadata = Z[4].metadata.copy()
# dataset_test5= Z[3].copy()

In [6]:
# dataset_train1.convert_to_dataframe()[0].to_csv("Huangrui/german/german_train1.csv",index=False)
# dataset_test1.convert_to_dataframe()[0].to_csv("Huangrui/german/german_test1.csv",index=False)
# dataset_train2.convert_to_dataframe()[0].to_csv("Huangrui/german/german_train2.csv",index=False)
# dataset_test2.convert_to_dataframe()[0].to_csv("Huangrui/german/german_test2.csv",index=False)
# dataset_train3.convert_to_dataframe()[0].to_csv("Huangrui/german/german_train3.csv",index=False)
# dataset_test3.convert_to_dataframe()[0].to_csv("Huangrui/german/german_test3.csv",index=False)
# dataset_train4.convert_to_dataframe()[0].to_csv("Huangrui/german/german_train4.csv",index=False)
# dataset_test4.convert_to_dataframe()[0].to_csv("Huangrui/german/german_test4.csv",index=False)
# dataset_train5.convert_to_dataframe()[0].to_csv("Huangrui/german/german_train5.csv",index=False)
# dataset_test5.convert_to_dataframe()[0].to_csv("Huangrui/german/german_test5.csv",index=False)


In [18]:
protected_attribute_used = 2
K = 2
#     dataset_orig = GermanDataset()
if protected_attribute_used == 1:
    privileged_groups = [{'sex': 1}]
    unprivileged_groups = [{'sex': 0}]
else:
    privileged_groups = [{'age': 1}]
    unprivileged_groups = [{'age': 0}]

if protected_attribute_used == 1:
    dataset_orig_train= GermanDataset(path="./Huangrui/german/german_train{}.csv".format(K),protected_attribute_names=['sex'],
                 privileged_classes=[[1]])
    dataset_orig_test= GermanDataset(path="./Huangrui/german/german_test{}.csv".format(K),protected_attribute_names=['sex'],
                 privileged_classes=[[1]])
else:
    dataset_orig_train= GermanDataset(path="./Huangrui/german/german_train{}.csv".format(K),protected_attribute_names=['age'],
                 privileged_classes=[[1]])
    dataset_orig_test= GermanDataset(path="./Huangrui/german/german_test{}.csv".format(K),protected_attribute_names=['age'],
                 privileged_classes=[[1]])
import pickle
from sklearn.linear_model import Lasso
# 用部分 data训练 biased model
selected_dataset,_ = dataset_orig_train.split([0.6], shuffle=True)
_ =  dataset_orig_test
# Logistic regression classifier and predictions
scale_orig = StandardScaler()
X_train = scale_orig.fit_transform(selected_dataset.features)
y_train = selected_dataset.labels.ravel()
X_test = scale_orig.fit_transform(_.features)
y_test = _.labels.ravel()
lmod = Lasso(alpha = 0.001)
lmod.fit(X_train, y_train)
y_train_pred = lmod.predict(X_train)>0.5
y_test_pred = lmod.predict(X_test)>0.5
dataset_orig_train_pred = selected_dataset.copy(deepcopy=True)
dataset_orig_train_pred.labels = y_train_pred
dataset_orig_test_pred = _.copy(deepcopy=True)
dataset_orig_test_pred.labels = y_test_pred
print(np.mean(y_train_pred!=y_train))
print(np.mean(y_test_pred!=y_test))
metric_test = compute_metrics(_, dataset_orig_test_pred, 
                unprivileged_groups, privileged_groups)
metric_train = compute_metrics(selected_dataset, dataset_orig_train_pred,unprivileged_groups, privileged_groups)

0.2
0.27
Balanced accuracy = 0.6342
Statistical parity difference = -0.2615
Disparate impact = 0.6799
Average odds difference = -0.2543
Equal opportunity difference = -0.2443
Theil index = 0.1674
Balanced accuracy = 0.7538
Statistical parity difference = -0.3858
Disparate impact = 0.5118
Average odds difference = -0.2772
Equal opportunity difference = -0.2794
Theil index = 0.1237


In [20]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(800, 51)


#### Favorable and unfavorable labels

1.0 0.0


#### Protected attribute names

['age']


#### Privileged and unprivileged protected attribute values

[array([1.])] [array([0.])]


#### Dataset feature names

['month', 'credit_amount', 'investment_as_income_percentage', 'residence_since', 'age', 'number_of_credits', 'people_liable_for', 'sex', 'status=200+', 'status=<200', 'status=None', 'credit_history=Delay', 'credit_history=None/Paid', 'credit_history=Other', 'purpose=A40', 'purpose=A41', 'purpose=A410', 'purpose=A42', 'purpose=A43', 'purpose=A44', 'purpose=A45', 'purpose=A46', 'purpose=A48', 'purpose=A49', 'savings=500+', 'savings=<500', 'savings=Unknown/None', 'employment=1-4 years', 'employment=4+ years', 'employment=Unemployed', 'other_debtors=A101', 'other_debtors=A102', 'other_debtors=A103', 'property=A121', 'property=A122', 'property=A123', 'property=A124', 'installment_plans=A141', 'installment_plans=A142', 'installment_plans=A143', 'housing=A151', 'housing=A152', 'housing=A153', 'skill_level=A171', 'skill_level=A172', 'skill_level=A173', 'skill_level=A174', 'telephone=A191', 'telephone=A192', 'foreign_worker=A201', 'foreign_worker=A202']


In [21]:
#save model
bmodels = {}
bmodels['clf'] = lmod
pickle.dump(bmodels, open("experiments/german"+str(K)+'_age_bmodel.pkl','wb'))